<h1 style="color: orange; font-size: 48px;">YOLO</h1>
<hr>


 <font color='orange'>1. Entrenamiento del modelo a partir del modelo base preentrenado</font>

 * Se entrena con el modelo base yolov8n.pt

 * Genera el modelo que ha aprendido especificamente las tareas de reconocimiento de moho, gracias a las imágenes del conjunto de entrenamiento.

 * Crea un modelo llamado yolov8_mold_finetune que es el que se usará para testear el modelo. En la ruta: C:\Users\adina\OneDrive\Documentos\TFM\new\yolo_640\runs\train\yolov8_mold_finetune\weights se encuentran los pesos. best.p >> el mejor modelo, YOLO para de entrenar. guardo los pesos de los mejores resultados. last.pt >> es el correspondiente de la última ejecución, no se debe utilizar, salvo para si queremos seguir entrenando, es como un 'save´file', guarda la partida actual.


In [ ]:
# Entrenaimento, fine tuning modelo cn carpeta train 3XX imagenes >>>>> OK
from ultralytics import YOLO

# 1. Carga el modelo base preentrenado
model = YOLO("yolov8n.pt")  

# 2. Lanza el entrenamiento / fine‑tuning
model.train(
    data=r"C:\Users\adina\OneDrive\Documentos\TFM\new\split\yolo_split\data.yaml",
    epochs=50,
    imgsz=640,
    batch=4,
    project=r"C:\Users\adina\OneDrive\Documentos\TFM\new\yolo_640\runs\train",
    name="yolov8_mold_finetune"
)

 <font color='orange'>2. Prueba del modelo fine-tunning best.pt</font>

 * Se prueba el modelo con las muestras del conjunto de test. Se muestran las imágene sy se compara con las etiquetas.

 * Se hace un overlay de las bounding boxes detectadas con las reales. Las reales son las de color verse, las detectadas y APTAS son azules, las NO APTAS son rojas. Si max_iou ≥ iou_thresh, se cuenta como TP y se dibuja en azul; si no, como FP en rojo.

 * Cálculo de FN: FN = número de GT – TP.

 * Calcula IOU: calcular_iou(boxA, boxB) recibe dos cajas en formato [x1, y1, x2, y2] y devuelve su Intersection over Union.

 * Fija un umbral de IoU (iou_thresh = 0.15, antes 0.45) para decidir si una predicción se considera correcta. El hiperparámetro iou_thresh se modifica visualmente, viendo las muestras.

 * Se imprimen las imágenes y las métricas por imagen. Luego, se muestran las métricas resultado global.

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Parámetros de entrada
input_dir = r"C:\Users\adina\OneDrive\Documentos\TFM\new\dataset_etiquetado_split\yolo\data\test"
label_dir = r"C:\Users\adina\OneDrive\Documentos\TFM\new\dataset_etiquetado_split\yolo\data\test"
model_path = r"C:\Users\adina\OneDrive\Documentos\TFM\new\yolo_640\runs\train\yolov8_mold_finetune\weights\best.pt"
iou_thresh = 0.15

def calcular_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    if interArea == 0:
        return 0.0
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    return interArea / float(boxAArea + boxBArea - interArea)

model = YOLO(model_path)

# Acumuladores globales
TP_total = 0
FP_total = 0
FN_total = 0

# Obtener lista de imágenes
imagenes = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

for imagen_nombre in imagenes:
    img_path = os.path.join(input_dir, imagen_nombre)
    lbl_path = os.path.join(label_dir, os.path.splitext(imagen_nombre)[0] + ".txt")

    img = cv2.imread(img_path)
    if img is None:
        print(f"[ERROR] No se pudo cargar {imagen_nombre}")
        continue
    h, w = img.shape[:2]

    gt_boxes = []
    if os.path.exists(lbl_path):
        with open(lbl_path, 'r') as f:
            for line in f:
                cls, xc, yc, bw, bh = map(float, line.split())
                x1 = int((xc - bw/2) * w)
                y1 = int((yc - bh/2) * h)
                x2 = int((xc + bw/2) * w)
                y2 = int((yc + bh/2) * h)
                gt_boxes.append([x1, y1, x2, y2])
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    else:
        print(f"[WARN] No se encontró GT para {imagen_nombre}")

    results = model.predict(img_path, imgsz=640, conf=0.20, verbose=False)
    preds = []
    for box in results[0].boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        preds.append([x1, y1, x2, y2])

    TP = 0
    FP = 0
    for p in preds:
        ious = [calcular_iou(p, g) for g in gt_boxes]
        max_iou = max(ious) if ious else 0.0
        if max_iou >= iou_thresh:
            TP += 1
            color = (255, 0, 0)  # azul
        else:
            FP += 1
            color = (0, 0, 255)  # rojo
        cv2.rectangle(img, (p[0], p[1]), (p[2], p[3]), color, 4)
        cv2.putText(img, f"{max_iou:.2f}", (p[0], p[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    FN = len(gt_boxes) - TP

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall    = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1        = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    accuracy  = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0

    # Acumular globales
    TP_total += TP
    FP_total += FP
    FN_total += FN

    # Mostrar métricas por imagen
    print(f"\nImagen: {imagen_nombre}")
    print(f"TP: {TP}, FP: {FP}, FN: {FN}")
    print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1: {f1:.2f}")

    # Mostrar imagen
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 8))
    plt.imshow(img_rgb)
    plt.title(f"{imagen_nombre} — GT (verde), Pred (azul=TP, rojo=FP), IoU≥{iou_thresh}")
    plt.axis('off')
    plt.show()

# ==========================
#      MÉTRICAS GLOBALES
# ==========================
precision_total = TP_total / (TP_total + FP_total) if (TP_total + FP_total) > 0 else 0
recall_total    = TP_total / (TP_total + FN_total) if (TP_total + FN_total) > 0 else 0
f1_total        = 2 * precision_total * recall_total / (precision_total + recall_total) if (precision_total + recall_total) > 0 else 0
accuracy_total  = TP_total / (TP_total + FP_total + FN_total) if (TP_total + FP_total + FN_total) > 0 else 0

print("\n==============================")
print("   MÉTRICAS GLOBALES FINALES")
print("==============================")
print(f"TP total: {TP_total}, FP total: {FP_total}, FN total: {FN_total}")
print(f"Accuracy: {accuracy_total:.2f}")
print(f"Precision: {precision_total:.2f}")
print(f"Recall: {recall_total:.2f}")
print(f"F1 Score: {f1_total:.2f}")


 <font color='orange'>3. Inferencia sobre el modelo</font>

 * Umbral mínimo de confianza >> Reducirlo puede lograr más detecciones, pero también más FP
 * IOU  ajusta el nivel de aceptación de solapamiento de deteccciones. Se probó con 0.45 (resultados no válidos) y 0.15 - OK


 Resultados >> Confianza 0.25 IOU 0.15:

 | Métrica     | Valor  |
|-------------|--------|
| TP total    | 567    |
| FP total    | 106    |
| FN total    | 167    |
| Accuracy    | 0.68   |
| Precision   | 0.84   |
| Recall      | 0.77   |
| F1 Score    | 0.81   |

 Resultados >> Confianza 0.15 IOU 0.15:

 | Métrica     | Valor  |
|-------------|--------|
| TP total    | 915    |
| FP total    | 266    |
| FN total    | -181   |
| Accuracy    | 0.92   |
| Precision   | 0.77   |
| Recall      | 1.25   |
| F1 Score    | 0.96   |


 Resultados >> Confianza 0.25 IOU 0.45:

| Métrica     | Valor  |
|-------------|--------|
| TP total    | 441    |
| FP total    | 232    |
| FN total    | 293    |
| Accuracy    | 0.46   |
| Precision   | 0.66   |
| Recall      | 0.60   |
| F1 Score    | 0.63   |


 Resultados >> Confianza 0.20 IOU 0.15:

| Métrica     | Valor  |
|-------------|--------|
| TP total    | 706    |
| FP total    | 155    |
| FN total    | 28     |
| Accuracy    | 0.79   |
| Precision   | 0.82   |
| Recall      | 0.96   |
| F1 Score    | 0.89   |
